## A tweak to argsort()

* Recall argsort() from last time:

In [5]:
def argsort(input):
    return sorted(range(len(input)), key=input.__getitem__)

B = ["TAGACAT", "AGACAT", "GACAT", "ACAT", "CAT", "AT", "T"]
print(argsort(B))

[3, 1, 5, 4, 2, 6, 0]


## Constructing a Suffix Array

In [1]:
def suffixArray(string):
    return [i for i in sorted(range(len(string)), key=lambda x: string[x:])]

t = "amanaplanacanalpanama"
sa = suffixArray(t)
print(sa)
for i in sa:
    print("%2d: %s" % (i, t[i:]))

[20, 9, 13, 18, 0, 7, 11, 16, 2, 4, 10, 6, 14, 19, 1, 8, 12, 17, 3, 15, 5]
20: a
 9: acanalpanama
13: alpanama
18: ama
 0: amanaplanacanalpanama
 7: anacanalpanama
11: analpanama
16: anama
 2: anaplanacanalpanama
 4: aplanacanalpanama
10: canalpanama
 6: lanacanalpanama
14: lpanama
19: ma
 1: manaplanacanalpanama
 8: nacanalpanama
12: nalpanama
17: nama
 3: naplanacanalpanama
15: panama
 5: planacanalpanama


## Searching a Suffix Array

* Searching a sorted list requires $O(log(m))$ comparisons using ***binary search***
* Each comparision is over *n* symbols of the pattern
* Thus, searching is $O(n log(m))$

In [7]:
def findFirst(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:] < pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

first = findFirst("an", t, sa)
print(t)
print(first, sa[first], t[sa[first]:])

amanaplanacanalpanama
5 7 anacanalpanama


## Finding all Occurences

A variant to binary search which finds the last occurence of a pattern rather than the first. Only difference, uses "<=" instead of "<", but needs to trim string comparison to test for equality.

In [8]:
def findLast(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:suffixarray[middle]+len(pattern)] <= pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

print(t)
last = findLast("an", t, sa)
print(first, last)
for suffix in sa[first:last]:     # recall "first" was found on the previous slide
    print("%3d: %s" % (suffix, t[suffix:]))
print(last - first, "times")

amanaplanacanalpanama
5 9
  7: anacanalpanama
 11: analpanama
 16: anama
  2: anaplanacanalpanama
4 times


## Longest repeated substring?

* Given a suffix array, we can compute a *helper* function, call the **Longest Common Prefix**, LCP

In [9]:
def computeLCP(text, suffixarray):
    m = len(text)
    lcp = [0 for i in range(m)]
    for i in range(1,m):
        u = suffixarray[i-1]
        v = suffixarray[i]
        n = 0
        while text[u] == text[v]:
            n += 1
            u += 1
            v += 1
            if (u >= m) or (v >= m):
                break
        lcp[i] = n
    return lcp

lcp = computeLCP(t, sa)

print("SA,LCP,Suffix")
for i, j in enumerate(sa):
    print("%2d: %2d %s" % (j, lcp[i], t[j:]))

SA,LCP,Suffix
20:  0 a
 9:  1 acanalpanama
13:  1 alpanama
18:  1 ama
 0:  3 amanaplanacanalpanama
 7:  1 anacanalpanama
11:  3 analpanama
16:  3 anama
 2:  3 anaplanacanalpanama
 4:  1 aplanacanalpanama
10:  0 canalpanama
 6:  0 lanacanalpanama
14:  1 lpanama
19:  0 ma
 1:  2 manaplanacanalpanama
 8:  0 nacanalpanama
12:  2 nalpanama
17:  2 nama
 3:  2 naplanacanalpanama
15:  0 panama
 5:  1 planacanalpanama


## Key Idea behind the BWT

* Sorting Cyclical Suffixes (say that 3-times fast)
<pre>
         "Cyclical Suffixes"         "Sorted Cyclical Suffixes" 
              tarheel$                    $tarhee<span style="color:blue;">l</span>
              arheel$t                    arheel$<span style="color:blue;">t</span>
              rheel$ta                    eel$tar<span style="color:blue;">h</span>
              heel$tar                    el$tarh<span style="color:blue;">e</span>
              eel$tarh                    heel$ta<span style="color:blue;">r</span>
              el$tarhe                    l$tarhe<span style="color:blue;">e</span>
              l$tarhee                    rheel$t<span style="color:blue;">a</span>
              $tarheel                    tarheel<span style="color:blue;">&dollar;</span>
</pre>

* The BWT of "tarheels" is the last column of the sorted cyclical suffixes "ltherea&dollar;"
* Notice that the sorted cyclical suffixes have a lot in common with a suffix array.
* The BWT is just the "predecessor symbol of these suffixes", where "&dollar;" precedes the first symbol

In [10]:
# making cyclical suffixes
t="DDND$DANAAADA"
print([t[i:]+t[:i] for i in range(len(t))])

['DDND$DANAAADA', 'DND$DANAAADAD', 'ND$DANAAADADD', 'D$DANAAADADDN', '$DANAAADADDND', 'DANAAADADDND$', 'ANAAADADDND$D', 'NAAADADDND$DA', 'AAADADDND$DAN', 'AADADDND$DANA', 'ADADDND$DANAA', 'DADDND$DANAAA', 'ADDND$DANAAAD']


## BWT in Python
Straightforward implementation based on the definition (there are faster construction methods)

In [11]:
def BWT(t):
    # create a sorted list of all cyclic suffixes of t
    rotation = sorted([t[i:]+t[:i] for i in range(len(t))])
    # concatenate the last symbols from each suffix
    return ''.join(r[-1] for r in rotation)

print(BWT("banana$"))
print(BWT("amanaplanacanalpanama$"))
print(BWT("abananaban$"))

annb$aa
amnnn$lcpmnapaaaaaaala
nn$bnbaaaaa


## BWT from a Suffix Array

* It is even simpler to compute the BWT from a Suffix Array
* Finds each suffix's "predecessor" symbol

In [12]:
def BWTfromSuffixArray(text, suffixarray):
    return ''.join(text[i-1] for i in suffixarray)
    
newt = 'DDND$DANAAADA'
sa = suffixArray(newt)
print(newt)
print(sa)
print(BWTfromSuffixArray(newt, sa))

DDND$DANAAADA
[4, 12, 8, 9, 10, 6, 3, 11, 5, 0, 1, 7, 2]
DDNAADNA$ADAD


## Inverse BWT in Python

In [13]:
def inverseBWT(bwt):
    # initialize the table from t
    table = ['' for c in bwt]
    for j in range(len(bwt)):
        #insert the BWT as the first column
        table = sorted([c+table[i] for i, c in enumerate(bwt)])
    #return the row that ends with ‘$’
    return table[bwt.index('$')]

print(inverseBWT("ltherea$"))
print(inverseBWT("amnnn$lcpmnapaaaaaaala"))
print(inverseBWT("annb$aa"))
print(inverseBWT("nn$bnbaaaaa"))

ADNADADANDAD$
amanaplanacanalpanama$
banana$
abananaban$


## Constructing the FM-index

In [11]:
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]  # a list of dictionaries
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].items()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

bwt = "annb$aa"
FM, Offset = FMIndex(bwt)
print("BWT %2s,%2s,%2s,%2s" % tuple([symbol for symbol in sorted(Offset.keys())]))
for i, row in enumerate(FM):
    data = [bwt[i]+':' if i < len(bwt) else '']+[row[symbol] for symbol in sorted(row.keys())]
    print("%3s %2d,%2d,%2d,%2d" % tuple(data))
print()
print([(sym, Offset[sym]) for sym in sorted(Offset.keys())])

BWT  $, a, b, n
 a:  0, 0, 0, 0
 n:  0, 1, 0, 0
 n:  0, 1, 0, 1
 b:  0, 1, 0, 2
 $:  0, 1, 1, 2
 a:  1, 1, 1, 2
 a:  1, 2, 1, 2
     1, 3, 1, 2

[('$', 0), ('a', 1), ('b', 4), ('n', 5)]


## Suffix Recovery

* What is the suffix array entry corresponding to BWT index *i*?
   - Start at *i* and repeatedly find predecessors until *i* is reached again

* To find the *original* string, just start with *i = 0*, the '$' index

In [39]:
def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

# recall that the FM-index that we built was "annb$aa", the BWT of "banana$"
for i in range(len(bwt)):
    print(i, recoverSuffix(i, bwt, FM, Offset), bwt[i])

0 $banana a
1 a$banan n
2 ana$ban n
3 anana$b b
4 banana$ $
5 na$bana a
6 nana$ba a


## BWT string search in Python

* One of the simplest methods we've seen for searching

In [40]:
def findBWT(pattern, FMIndex, Offset):
    lo = 0
    hi = len(FMIndex) - 1
    for symbol in reversed(pattern):
        lo = Offset[symbol] + FMIndex[lo][symbol]
        hi = Offset[symbol] + FMIndex[hi][symbol]
    return lo, hi

print(findBWT("ana", FM, Offset))
print(findBWT("ban", FM, Offset))
print(findBWT("ann", FM, Offset))

(2, 4)
(4, 5)
(4, 4)
